In [1]:
import networkx as nx
import random as rd
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter
import math
import pandas as pd

In [2]:
##################### ACTIVITY_DISTRIBUTION #########################

def Activity_Distribution(x, gamma):
    return x**(-gamma)

##################### PROBABILITY_DISTRIBUTION_ACTIVITY #############

def Probability_Distribution_Activity( gamma, eps1, eps2):
    if eps1 == 0:
        print("Error! Eps1 = 0!\n")
        return 0

    den = 10000

    prob = []
    step = (eps2 - eps1)/den
    # Create Norm of Distribution
    norm = 0
    for i in range (den+1):
        norm += Activity_Distribution((eps1 + i*step), gamma)

    # Create probabilities
    for i in range (den+1): 
        val = Activity_Distribution((eps1 + i*step), gamma)
        prob.append(val/norm)
    return prob

##################### ASSING_ACTIVITY ###############################

def Assign_Activity(prob, eps1, eps2):
    len_arr = len(prob)
    step = (eps2 - eps1)/10000
    arr = np.arange(eps1, eps2+step/2, step)
    rand = np.random.choice(arr, 1, p=prob)
    return rand

#################### INITIALIZE_NETWORK #############################

def Initialize_Network(N, T, gamma, eps1, eps2):

    # Create Graph with N not connected nodes
    G = nx.Graph()
    add_nod = np.arange(0, N, 1)
    G.add_nodes_from(add_nod)

    # Create probability distribution
    prob = Probability_Distribution_Activity(gamma, eps1, eps2)

    # Initialize Activity and Opinions of Nodes
    for i in G.nodes:
        G.nodes[i]['activity'] = Assign_Activity(prob, eps1, eps2)
        opinion = []
        for j in range (1,T+1):
            opinion.append(np.random.normal(0, np.sqrt(2.5)))
        G.nodes[i]['opinion'] = opinion
    
    #for i in G.nodes:
    #print("Aktivität node 0:", G.nodes[0]['activity'], "\n")
    return G

#################### PROBABILITY_DISTRIBUTION_OPINION ###############

def Probability_Distribution_Opinion(G, numb_node, T, beta, Phi):
    prob = []

    # Calculate Norm
    norm = 0
    for i in G.nodes:
        #sp = 0
        distance = 0
        if i != numb_node: 
            # numb_node is the number of the selected node
            for u in range (T):
                for v in range (T):
                    distance += (G.nodes[i]['opinion'][u] - G.nodes[numb_node]['opinion'][u]) * (G.nodes[i]['opinion'][v] - G.nodes[numb_node]['opinion'][v]) * Phi[u][v]
            norm += np.sqrt(distance)**(-beta)
        

    # Calculate Probability of contact
    for i in G.nodes: 
        if i != numb_node:
            #sp = 0
            distance = 0
            for u in range (T):
                for v in range (T):
                    distance += (G.nodes[i]['opinion'][u] - G.nodes[numb_node]['opinion'][u]) * (G.nodes[i]['opinion'][v] - G.nodes[numb_node]['opinion'][v]) * Phi[u][v]
            prob.append(np.sqrt(distance)**(-beta) / norm)
        else:
            prob.append(0)
    return prob

#################### PLOT_OPINION_DISTRIBUTION_2D ###################

def Plot_Opinion_Distribution_2d(G):
    op1_list = []
    op2_list = []
    for i in G.nodes:
        op1_list.append(G.nodes[i]['opinion'][0])
        op2_list.append(G.nodes[i]['opinion'][1])
    op1_counts = Counter(op1_list)
    op2_counts = Counter(op2_list)

    fig, ax = plt.subplots(1,3)
    ax[0].set_title("Topic 1")
    ax[0].set_xlabel("Opinion")
    ax[0].set_ylabel("# of Opinions on Topic 1")
    ax[0].hist(op1_list, bins=120)
    ax[1].set_title("Topic 2")
    ax[1].set_xlabel("Opinion")
    ax[1].set_ylabel("# of Opinions on Topic 2")
    ax[1].hist(op2_list, bins=120)
    ax[2].set_title("Opinion-Space")
    ax[2].set_xlabel("Opinion 1")
    ax[2].set_ylabel("Opinion 2")
    ax[2].scatter(op1_list, op2_list, marker = '2')
    plt.show()
    
#################### ODEs ###########################################

def ODEs(N, T, A, K, alpha, Phi, current_opinions, opinions_step):
    # Determine social influence to return the differential. current_opinions and opinions_step
    # are arrays of the form [..., [agent_i_op1, agent_i_op2], [agent_i+1_op1, agent_i+1_op2], ...]
    influence = np.zeros((N,T))
    for i in range (N):
        for o in range (T):
            # Go through connected agents of i
            for j in A[i]:
                influence[i][o] += np.tanh(alpha * np.dot(Phi, current_opinions[j])[o])

    #return differential
    dxdt = -opinions_step + K * influence
    return dxdt

#################### RK4 #############################################

def RK4(G, T, A, K, alpha, Phi, dt):
    N = G.number_of_nodes()
    # Save current opinions
    current_opinions = np.array([G.nodes[i]['opinion'] for i in range (N)])
    
    # Calculate ks
    k1 = dt * ODEs(N, T, A, K, alpha, Phi, current_opinions, current_opinions)
    k2 = dt * ODEs(N, T, A, K, alpha, Phi, current_opinions, current_opinions + 0.5 * k1)
    k3 = dt * ODEs(N, T, A, K, alpha, Phi, current_opinions, current_opinions + 0.5 * k2)
    k4 = dt * ODEs(N, T, A, K, alpha, Phi, current_opinions, current_opinions + k3)

    # Calculate total change and update opinions
    k = 1/6 * (k1 + 2*k2 + 2*k3 + k4)
    current_opinions += k
    for i in range (N):
        G.nodes[i]['opinion'] = current_opinions[i]
    




##################### OPINION_DYNAMICS ################################

def Opinion_Dynamics(N, T, m, K, alpha, beta, gamma, Phi, eps1, eps2, runtime_net, runtime_op, dt, filename):
    # Safety errors
    if eps1 <= 0 or eps2 > 1:
        print("ERROR! [eps1, eps2] must be included in (0,1]\n")
        return 0
    if T <= 0:
        print("ERROR! Number of opinions has to be atleast 1!\n")
        return 0
    if Phi == 1:
        if T != 1:
            print("ERROR! Matrix Phi does not have T rows and columns!\n")
            return 0
    else:
        if len(Phi) != T and len(Phi[0]) != T:
            print("ERROR! Matrix Phi does not have T rows and columns!\n")
            return 0

    # Create array to save opinions of all agents after every network-iteration
    save = np.zeros((runtime_net * T + 1, N))

    G = Initialize_Network(N, T, gamma, eps1, eps2)

    # Save activities of agents in first row
    for i in range (N):
        save[0][i] += G.nodes[i]['activity']

    # Perform Iterations until runtime is reached
    iteration_net = 0
    while iteration_net < runtime_net:

        # Form adjecency list for timestep
        A = []
        for i in range (N):
            A.append([])

        for i in G.nodes:
            # Go through nodes and possibly activate them
            rand = np.random.uniform(0,1)
            if rand <= G.nodes[i]['activity']:
                # Pick m other nodes randomly. No exception of i needed, since
                # P_D_O excludes i already
                prob = Probability_Distribution_Opinion(G, i, T, beta, Phi)
                picks = np.random.choice(G.nodes, m, replace=False, p=prob)
                #Update adjacency list: append nodes j to i´s place and vice versa
                for j in picks:
                    A[i].append(j)
                    A[j].append(i)
                    
        #if (iteration_net % 50) == 0:
        #    Plot_Opinion_Distribution_2d(G)
        #print("iteration ", iteration, ": ",  G.nodes[0]['opinion'])
        # Calculate the influence of the nodes on eachother in a random order
        # Create random order of nodes
        iteration_op = 0
        while iteration_op < runtime_op:
            # Update opinions via Runge-Kutta 4
            RK4(G, T, A, K, alpha, Phi, dt)
            # increase iteration
            iteration_op += 1
        
        # Save opinions of agents
        for i in range (N):
            for j in range(T):
                save[(T*iteration_net)+j+1][i] += G.nodes[i]['opinion'][j]
        
        iteration_net += 1

    # Export save to csv
    # Create column indices (optional)
    nodes_ind = []
    for i in range (N):
        nodes_ind.append(f'Node_{i}')
    # Create row indices (optional)
    op_ind = []
    op_ind.append('activity')
    for i in range (runtime_net):
        for j in range (T):
            op_ind.append(f'iteration {i} op {j}')
    # Create Dataframe and convert it to .csv
    df = pd.DataFrame(save, columns = nodes_ind, index = op_ind)
    df.to_csv (f'D:\Daten mit Änderungen\Physik\Bachelorarbeit\Generated_Data\{filename}.csv', index = False, header = False)

    return np.var(save[N*T])
        
            



In [5]:
N = 1000 # 1000 nodes should be the minimum, below initial fluctuations take over and polarization becomes instable
T = 2
m = math.floor((N+50)/100)
K = 3
alpha = 3
beta = 3
gamma = 2.1
Phi = [[1,0],[0,1]]
eps1 = 0.01
eps2 = 1
runtime_net = 1000 # Stability is reached from about 500 iterations
runtime_op = 1
step = 0.01
print("m: ", m)
print("Phi: ", Phi[0][0], Phi[0][1], Phi[1][0], Phi[1][1])
File_Names= ["a3_b3_g2.1_e2_1.0_16", "a3_b3_g2.1_e2_1.0_17"]
#variance = np.zeros((7,4))

m:  10
Phi:  1 0 0 1


In [6]:
for j in range (len(File_Names)):
    Opinion_Dynamics(N, T, m, K, alpha, beta, gamma, Phi, eps1, eps2, runtime_net, runtime_op, step, File_Names[j])